In [216]:
using DifferentialEquations, Plots

function bump(x, s, c)
    
    if x <= c-s || x >= c+s
        return 0
    else
        return 1/2 * exp(-(s^2/(s^2-(x-c)^2)))
    end
    
    #return 0
end

function intialFunc(x, b, xf, c, s)
    b + (1-b)/(1 + exp(1/c * (x - xf))) + s
end

intialFunc (generic function with 1 method)

In [199]:
function system!(du, u, p, t)
    D, alpha, dx, bscale, bcenter = p

    function phi(i)
        u[i] + bump(dx*(i-2), bscale, bcenter)
    end
    function interp(i)
        1/2 * (u[i]^3 + u[i+1]^3)
    end
    function f1(i)
        1/(dx^2) * (interp(i-1)*(phi(i-1) - phi(i)) + interp(i)*(phi(i+1) - phi(i)))
    end
    function f2(i)
        (1/dx^4) * (interp(i-1)*(phi(i-2) - 3*phi(i-1) + 3*phi(i) - phi(i+1)) + interp(i)*(-phi(i-1) + 3*phi(i) - 3*phi(i+1) + phi(i+2)))
    end
    function f3(i)
        (1/(2*dx)) * (u[i+1]^3 - u[i-1]^3)
    end

    du[1] = 0
    du[2] = 0
    du[end-1] = 0
    du[end] = 0
    for i in 3:length(u)-2
        du[i] = D*cos(alpha)*f1(i) - f2(i) - sin(alpha)*f3(i)
    end
    return du
end

system! (generic function with 1 method)

In [217]:

# X Domain
nx = 800
Lx = 40
dx = Lx/nx
x = 0.0:dx:Lx

# Bump function params
bcenter = 20.0
bscale = 5.0
s = bump.(x, bscale, bcenter)

# Initial condition params
b = 0.1
xf = 5
ic = intialFunc.(x, b, xf, 0.5, s)
pushfirst!(ic, 1.0)
push!(ic, b)

D = 1
alpha = pi/2

p = (D, alpha, dx, bscale, bcenter)
tspan = (0.0, 30.0)
prob = ODEProblem(system!, ic, tspan, p, saveat=0.5)
sol = solve(prob, alg=Rosenbrock23())

retcode: Success
Interpolation: 1st order linear
t: 61-element Vector{Float64}:
  0.0
  0.5
  1.0
  1.5
  2.0
  2.5
  3.0
  3.5
  4.0
  4.5
  ⋮
 26.0
 26.5
 27.0
 27.5
 28.0
 28.5
 29.0
 29.5
 30.0
u: 61-element Vector{Vector{Float64}}:
 [1.0, 0.9999591419181678, 0.9999548450517846, 0.9999500963277494, 0.999944848234345, 0.9999390482653423, 0.9999326383952405, 0.9999255544994335, 0.999917725713533, 0.9999090737254831  …  0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
 [1.0000000000000044, 0.9999591419181607, 0.9999061784268275, 0.9998435692449358, 0.999773796367686, 0.9996993676319653, 0.9996228194896495, 0.9995467189689486, 0.9994736648025851, 0.9994062877005262  …  0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
 [1.000000000000001, 0.99995914191818, 0.9998893657410064, 0.9997930982471185, 0.9996727981409279, 0.9995309660981203, 0.9993701540736218, 0.999192973864215, 0.9990021049135298, 0.9988003013429432  …  0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
 [0.9999999999999895

In [234]:

using Dates
dt = string(Dates.format(now(), "YYYYmmdd-HHMMSS"))
path = "../runs/$dt"
dt = floor(Int8, 2/0.5)

using LaTeXStrings
plot_font = "Computer Modern"
default(fontfamily=plot_font, grid=false)

anim = @animate for i = 1:size(sol.u)[1]
    plot(x, s, color="black", xlabel=L"x", ylabel=L"h")
    plot!(x, sol.u[i][2:end-1], legend=false, ylims=(0,1.5), xlims=(0, Lx))
end

temp = sol.u[1:dt:end]
for i in 1:length(temp)
    temp[i] = temp[i][2:end-1]
end
temp
size(temp[1])

plt = plot(x, s, dpi=200, xlabel=L"x", ylabel=L"h", legend=false, color="black",
        title="Fluid profile at " * L"\delta t = 2" * " intervals", ylims=(0, Inf), xlims=(0, Lx))
for i = 1:length(temp)
    plot!(plt, x, temp[i])
end

using CSV, DataFrames
mkpath(path)
gif(anim, path * "/flow.gif", fps = 5)
savefig(plt, path * "/plt.png")
df = DataFrame([sol.t, sol.u], :auto)
#=
Format: First Col -> Time
        Second Col -> h_i
=#
CSV.write(path * "/data.csv", df, header = false)

┌ Info: Saved animation to 
│   fn = /Users/bsamineni/Documents/research_projects/acoustics_thin_film/notes/runs/20220223-213259/flow.gif
└ @ Plots /Users/bsamineni/.julia/packages/Plots/LI4FE/src/animation.jl:114


"../runs/20220223-213259/data.csv"